In [37]:
import os
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import mixed_precision as mp
from ultralytics import YOLO
from ultralytics.data.converter import convert_coco
from roboflow import Roboflow
from PIL import Image
from datetime import datetime
from dotenv import load_dotenv

In [2]:
# use gpu if available
physical_devices = tf.config.list_physical_devices('GPU')
print("Physical devices:", physical_devices)
if physical_devices:
    print("GPU available, enabling mixed precision training")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    policy = mp.Policy('mixed_float16')
    mp.set_global_policy(policy)
else:
    print("GPU not available, using CPU")

print("PyTorch CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

load_dotenv()

Physical devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU available, enabling mixed precision training
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060, compute capability 8.6
PyTorch CUDA available: True
NVIDIA GeForce RTX 3060


True

In [55]:
def download_data(Roboflow, folderpath='datasets/court_segmentation'):
    # https://universe.roboflow.com/thesis-r4nbe/volleyball-court-seg/dataset/8
    rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
    project = rf.workspace("thesis-r4nbe").project("volleyball-court-seg")
    version = project.version(8)
    dataset = version.download("coco-segmentation", location=folderpath)
    return dataset

def train_model(YOLO, tf, folderpath='datasets/court_segmentation'):
    model = YOLO('yolo11n-seg.pt')
    print(model)
    model.info()
    train_results = model.train(
        data=os.path.join(folderpath, 'data.yaml'),
        epochs=150,
        imgsz=640,
        batch=16,
        device=0 if tf.config.list_physical_devices('GPU') else 'cpu',
        workers=4,
        optimizer='Adam',
        lr0=0.01,
        lrf=0.01,
        patience=20,
        project="runs/segment/train",
        name="yolov11-court-seg-v2",
        save=True,
        save_period=10,
        exist_ok=True,
        verbose=True,
        plots=True,
        val=True,
    )
    return model, train_results

def evaluate_model(model):
    metrics = model.val()
    return metrics

def run_inference(model, image_path):
    results = model(image_path)
    results[0].show(mask=True)
    return results

def export_model(model):
    path = model.export(format="onnx")
    return path

In [ ]:
# make if statement, reload if tensorboard already loaded
# %reload_ext tensorboard
%load_ext tensorboard
%tensorboard --logdir runs/detect/train

In [54]:
dataset_folderpath = "datasets/court_segmentation"
dataset = download_data(Roboflow, folderpath=dataset_folderpath)

loading Roboflow workspace...
loading Roboflow project...
loading Roboflow project...


In [48]:
for split in ['train', 'valid', 'test']:
    convert_coco(
        labels_dir=os.path.join(dataset_folderpath, split),
        save_dir=os.path.join(dataset_folderpath, split),
        use_segments=True,
        cls91to80=False,
    )

Annotations D:\Coding\tobio\training\datasets\court_segmentation\train\_annotations.coco.json: 100% ━━━━━━━━━━━━ 6724/6724 2.3Kit/s 2.9s0.1s
COCO data converted successfully.
Results saved to D:\Coding\tobio\training\datasets\court_segmentation\train2
Annotations D:\Coding\tobio\training\datasets\court_segmentation\train\_annotations.coco.json: 100% ━━━━━━━━━━━━ 6724/6724 2.3Kit/s 2.9s
COCO data converted successfully.
Results saved to D:\Coding\tobio\training\datasets\court_segmentation\train2
Annotations D:\Coding\tobio\training\datasets\court_segmentation\valid\_annotations.coco.json: 100% ━━━━━━━━━━━━ 1922/1922 2.0Kit/s 0.9s0.1s
COCO data converted successfully.
Results saved to D:\Coding\tobio\training\datasets\court_segmentation\valid2
Annotations D:\Coding\tobio\training\datasets\court_segmentation\valid\_annotations.coco.json: 100% ━━━━━━━━━━━━ 1922/1922 2.0Kit/s 0.9s
COCO data converted successfully.
Results saved to D:\Coding\tobio\training\datasets\court_segmentation\valid2


In [61]:
model, train_results = train_model(YOLO, tf, folderpath=dataset_folderpath)

YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_runni

In [ ]:
metrics = evaluate_model(model)
# results = run_inference(model, "path/to/image.jpg")
onnx_path = export_model(model)